In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, date
import pandas as pd
import pickle

In [2]:
def getTodayIndex(year, month, day):
    keys = {'syear': year, 'smonth': month, 'sday': day} # 要傳給期貨交易所的 key
    r = requests.post("http://www.taifex.com.tw/chinese/3/3_1_1.asp", data = keys) # 抓取這個 key 回傳網頁的原始碼
    soup = BeautifulSoup(r.text , "lxml") # 把原始碼做整理
    # 這邊下面講
    soup_data = soup.select('table')[2].select('table')[1].select('td')
    return soup_data

In [3]:
#可以透過這個方式，知道抓下來的碼到底長什麼樣子，注意不要抓到例假日會跑出錯誤
getTodayIndex(2017,12,20)
print(getTodayIndex(2017,12,20))

[<td class="12bk"><div align="center">TX      
											  </div></td>, <td class="12bk"><div align="center">201712  </div></td>, <td align="right" class="12bk">10457</td>, <td align="right" class="12bk">10509</td>, <td align="right" class="12bk">10457</td>, <td align="right" class="12bk">10497</td>, <td align="middle" class="12green">
<font color="red">â²40</font>
</td>, <td align="middle" class="12green">
<font color="red">â² 0.38%</font>
</td>, <td align="right" class="12bk">10865</td>, <td align="right" class="12bk">88577</td>, <td align="right" class="12bk">99442</td>, <td align="right" class="12bk">
											-
											</td>, <td align="right" class="12bk">30811</td>, <td align="right" class="12bk">10497</td>, <td align="right" class="12bk">10498</td>, <td align="right" class="12bk">10877</td>, <td align="right" class="12bk">8856</td>, <td class="12bk"> <div align="center">TX      
											  </div></td>, <td class="12bk"> <div align="center">201801   
											  </div

In [4]:
def getData(start):

    # 先把 DataFrame 架構寫好
    data = {'date':[], 'open':[], 'high':[], 'low':[], 'settlement':[], 'volume':[], 'close':[], 'open_int':[], 'close_best_bid':[], 'close_best_ask':[]}

    # 這邊匯入我們想要的資料的起始日期
    day = date(start[0], start[1], start[2])
    # 這邊把停止日期設置在今天
    today = date.today()
    # 起始日至今日的總日數
    span = ((today - day).days) + 1
    # 因為是每日資料，所以間隔設為一日
    interval = timedelta(days=1)

    # 多定義一個把爬下來的資料放進 DataFrame 的方法
    def addtoData(column, index):
        try:
            data[column].append(int(soup_data[index].text))
        except Exception as e:
            print (e)
            data[column].append(np.nan)
     # 把整個 span 內的 soup_data 中的當日期指各項資料抓下來
    for _ in range(span):
        try:
            soup_data = getTodayIndex(day.year, day.month, day.day)
            data['date'].append(day)
            addtoData('open', 2)
            addtoData('high', 3)
            addtoData('low', 4)
            addtoData('settlement', 5)
            addtoData('volume', 8)
            addtoData('close', 11)
            addtoData('open_int', 12)
            addtoData('close_best_bid', 13)
            addtoData('close_best_ask', 14)
            print ('%d/%d/%d is loaded' %(day.year, day.month, day.day))
        except Exception as e:
            #print (e)
            print ('stock market was not open on %d/%d/%d' %(day.year, day.month, day.day))
        # 日期往後加一天
        day += interval

   # 把 data 放到 DataFrame 裡面
    df = pd.DataFrame(data)
    return df  

# 從 1998/7/21 開始爬
df = getData([2017,12,20])

print (df[['date','open','close','volume']])

invalid literal for int() with base 10: '\r\n\t\t\t\t\t\t\t\t\t\t\t-\r\n\t\t\t\t\t\t\t\t\t\t\t'
stock market was not open on 2017/12/20
2017/12/21 is loaded
2017/12/22 is loaded
stock market was not open on 2017/12/23
stock market was not open on 2017/12/24
2017/12/25 is loaded
2017/12/26 is loaded
2017/12/27 is loaded
2017/12/28 is loaded
2017/12/29 is loaded
stock market was not open on 2017/12/30
stock market was not open on 2017/12/31
stock market was not open on 2018/1/1
2018/1/2 is loaded
2018/1/3 is loaded
2018/1/4 is loaded
2018/1/5 is loaded
stock market was not open on 2018/1/6
stock market was not open on 2018/1/7
2018/1/8 is loaded
stock market was not open on 2018/1/9


ValueError: arrays must all be same length

In [ ]:
with open('TWfuture.pickle','wb') as f:
    pickle.dump(df, f)

In [ ]:
pickle_in = open('TWfuture.pickle', 'rb')
df = pickle.load(pickle_in)
df